In [123]:
# import the SimPy package 
#from simpy import initialize
#from SimPy.Simulation import *
import simpy
import random
import collections

In [152]:
class Source():
    """Evento llegada y distribuyo"""
    def __init__(self,env,numClientes,puestos,cont): 
        self.env = env
        self.num = numClientes
        self.puestos = puestos
        self.cont = cont
    
    def distribucion_llegada(self):
        for i in range(self.num):
            r = random.random()
            c = Customer(name = "clienteE%02d"%i)
            if r <= self.cont["PE"]:
                self.process = env.process(self.generar_emergencia(c))
            else: 
                self.process = env.process(self.generar_comercial(c))

    def generar_emergencia(self,c):
        env.process(c.llamada_emergencia(self.env,puestos=self.puestos,cont=self.cont))
        ia = random.uniform(self.cont["min"],self.cont["max"])
        #print("IA:", ia)
        yield self.env.timeout(ia)
    
    def generar_comercial(self,c):
        env.process(c.llamada_comercial(self.env,puestos=self.puestos,cont=self.cont))
        ia = random.uniform(self.cont["min"],self.cont["max"])
        #print("IA:", ia)
        yield self.env.timeout(ia)

In [138]:
class Customer():
    """Evento llegadas/llamadas de los clientes"""
    def __init__(self,name):
        self.name = name
        
    def llamada_emergencia(env,puestos,cont):
        arrive = env.now
        # Pido uso de los dos puestos, y me dan el primero que se libere
        comercial = puestos[0].request(priority=0)
        emergencia = puestos[1].request(priority=0)
        atendido = yield comercial | emergencia
        
        # Me atendieron por puestos comerciales?
        if comercial in atendido:
            cont["multitasking"]+=1
            wait = env.now-arrive #Tiempo de espera
            #wM.observe(wait) #Monitor de espera (sumatoria de espera en cola)
            #print "%7.4f %s: EMERGENCIA - Me atendieron en el puesto de comerciales despues de esperar %6.3f"%(env.now,self.name,wait)
            print(env.now,self.name,": EMERGENCIA - Me atendieron en un puesto comercial despues de esperar ",wait)
            #print(env.now,": EMERGENCIA - Me atendieron en un puesto comercial despues de esperar ",wait)
            ta= abs(random.normalvariate(cont["mu"],cont["sigma"]))
            yield env.timeout(ta)
            
            #yield hold,self,tib
            #yield release,self,counters[choice]
        else:
            wait = env.now-arrive #Tiempo de espera
            #wM.observe(wait) #Monitor de espera (sumatoria de espera en cola)
            #print "%7.4f %s: EMERGENCIA - Me atendieron en el puesto de emergencia despues de esperar %6.3f"%(env.now,self.name,wait)
            print(env.now,self.name,": EMERGENCIA - Me atendieron en un puesto de emergencia despues de esperar ",wait)
            #print(env.now,": EMERGENCIA - Me atendieron en un puesto de emergencia despues de esperar ",wait)
            ta= abs(random.normalvariate(cont["mu"],cont["sigma"]))
            yield env.timeout(ta)
            
            
    def llamada_comercial(env,puestos,cont):
        arrive = env.now
        # Pido uso de los dos puestos, y me dan el primero que se libere
        yield puestos[0].request(priority=1)
        wait = env.now-arrive #Tiempo de espera
        #wM.observe(wait) #Monitor de espera (sumatoria de espera en cola)
        print(env.now,self.name,": COMERCIAL - Me atendieron despues de esperar ",wait)
        #print(env.now,": COMERCIAL - Me atendieron despues de esperar ",wait)
        #print （"%7.4f %s: COMERCIAL - Me atendieron despues de esperar %6.3f",now(),self.name,wait)
        ta= abs(random.normalvariate(cont["mu"],cont["sigma"]))
        yield env.timeout(ta)
            
            #yield hold,self,tib
            #yield release,self,counters[choice]
        

Condiciones Iniciales

In [75]:
# Experimental data
maxNumber = 100 #clientes
maxTime = 14400 #segundos
#random.seed(RANDOM_SEED)
PE = 0.12 #Porcentaje Emergencia
PC = 0.88 #Porcentaje Comercial

# TA - distribucion normal
mu = 267.64 
sigma = 297.40

# IA - uniforme
IAMin = 1
IAMax = 60

# Contadores
ci = {"multitasking":0,
     "mu":mu,
     "sigma":sigma,
     "min":IAMin,
     "max":IAMax,
     "PE":PE,
     "PC":PC}

# Puestos de atencion - variables de control
print('Ingrese cantidad de puestos comerciales')
cantCom = int(input())
print('Ingrese cantidad de puestos de emergencia')
cantEmer = int(input())

Ingrese cantidad de puestos comerciales
10
Ingrese cantidad de puestos de emergencia
12


Modelo de Simulacion

In [153]:


env = simpy.Environment()
puestos = [simpy.PriorityResource(env,capacity=cantCom),
           simpy.PriorityResource(env,capacity=cantEmer)]
            # menor el numero priority, mayor es la prioridad
#wM = Monitor()
s = Source(env,maxNumber,puestos,ci)
env.process(s.distribucion_llegada())
env.run(until=maxTime)

ValueError: None is not a generator.

Results

In [ ]:
#result = wM.count(),wM.mean() #wM.mean --> promedio de espera en cola
print "Average wait for %3d completions was %5.3f minutes."% result

# Porcentaje de espera menor a 40 segundos
# Objetio: Mayor o igual al 90%

print('Cantidad de puestos comerciales',cantCom)
print('Cantidad de puestos de emergencia',cantEmer)

10
